# Instructions


Build and train a CNN based model to classify the image. 

You are allowed to follow the tutorials/examples from the internet, but you must have to know how each line of code works. 
 


Training Dataset: https://drive.google.com/file/d/1jODHcfo3PH1GeT7XpqYrt5uknc8rEWwJ/view?usp=sharing 


Test Dataset: https://drive.google.com/file/d/1ULZueNv2v2w5yHxabBX4OKMmvAiLb46D/view?usp=sharing


The dataset contains the only picture of dogs and cats. You will build a CNN-based model to predict a given image dog or cat.



**submission:**

Colab file name: studentId_cnn.ipynb

Share with iqbal@ndub.edu.bd

Submit this file as a colab notebook.


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# Import/read data

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Datasets/train'
valid_path = '/content/drive/MyDrive/Datasets/test'


# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 1s 0us/step


## Code for Traning

In [ ]:
import math

In [ ]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = True


# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/Datasets/train/*')


# our layers - you can add more if you want
x = Flatten()(inception.output)


prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)


# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Finding the test accuracy

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

FileNotFoundError: ignored

# Explorering the results 

In [ ]:
import matplotlib.pyplot as plt

# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')


# save it as a h5 file
from tensorflow.keras.models import load_model

model.save('model_inception.h5')


y_pred = model.predict(test_set)
y_pred
import numpy as np
y_pred = np.argmax(y_pred, axis=1)
y_pred

## 10 test images that the model predicted correctly




In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

model=load_model('model_resnet50.h5')
img_data

img=image.load_img('/content/drive/MyDrive/Datasets/Saif.jpg',target_size=(224,224))
x=image.img_to_array(img)
x

<img width=200px src="https://icatcare.org/app/uploads/2018/07/Thinking-of-getting-a-cat.png"> 
<img width=200px src="https://icatcare.org/app/uploads/2018/07/Thinking-of-getting-a-cat.png">
<img width=200px src="https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2019/12/03202400/Yellow-Labrador-Retriever.jpg">

##  10 test images that the model predicted incorrectly 


<img width=200px src="https://icatcare.org/app/uploads/2018/07/Thinking-of-getting-a-cat.png"> 

<img width=200px src="https://icatcare.org/app/uploads/2018/07/Thinking-of-getting-a-cat.png"><img width=200px src="https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2019/12/03202400/Yellow-Labrador-Retriever.jpg">

# Predicting my image



url: https://icatcare.org/app/uploads/2018/07/Thinking-of-getting-a-cat.png

my input image is<br>
<img width=200px src="https://icatcare.org/app/uploads/2018/07/Thinking-of-getting-a-cat.png">

the input image was predicted as "cat" with the probablity "0.93255"


# Discussion 

## Parameters
For example, 

Total Traning Images:

Activation functions used:

Number of layers used:

Number of nodes in the input layers:

Number of nodes in the output layers:

Training Time:



## New things learned from the project


## To complete this project, I struggled a lot with the followings
Write here what were the problems, and how did you solve?

# References

You must include every link that you take help from

In [ ]:
https://www.youtube.com/watch?v=chQNuV9B-Rw&list=PL_RLre6A5ze3WHt5RqvPGS6LhKexS2rdL&index=2
https://www.youtube.com/watch?v=FLf5qmSOkwU&list=PL_RLre6A5ze3WHt5RqvPGS6LhKexS2rdL&index=10


In [ ]:
https://github.com/krishnaik06/Tomato-Leaf-Disease-Prediction
https://github.com/balajisrinivas/Cats-Vs-Dogs-Prediction